## **3 кейс**

**Пользовательские сессии**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [1]:
!wget https://gist.github.com/Vs8th/9fcfa729905072c12b1e535c71565a68/raw/sessions.csv

--2025-03-30 07:04:21--  https://gist.github.com/Vs8th/9fcfa729905072c12b1e535c71565a68/raw/sessions.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9fcfa729905072c12b1e535c71565a68/raw/sessions.csv [following]
--2025-03-30 07:04:21--  https://gist.githubusercontent.com/Vs8th/9fcfa729905072c12b1e535c71565a68/raw/sessions.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880014 (1.8M) [text/plain]
Saving to: ‘sessions.csv’

sessions.csv        100%[===================>]   1.79M  --.-KB/s    in 0.07s   

2025-03-30 07:04:22 (24.5 MB/s) - ‘sessions.

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
import pandas as pd

df = pd.read_csv('sessions.csv')
df

,user_id,session
0,user0,"[""2023-05-06 20:43:45"", 1192, [[""page0"", ""2023..."
1,user1,"[""2023-05-03 15:39:47"", 1642, [[""page0"", ""2023..."
2,user1,"[""2023-05-16 21:24:30"", 506, [[""page0"", ""2023-..."
3,user1,"[""2023-05-09 01:41:35"", 1004, [[""page0"", ""2023..."
4,user1,"[""2023-05-26 05:31:50"", 984, [[""page0"", ""2023-..."
...,...,...
5278,user998,"[""2023-05-26 17:33:49"", 631, [[""page0"", ""2023-..."
5279,user998,"[""2023-05-15 23:21:39"", 1393, [[""page0"", ""2023..."
5280,user998,"[""2023-05-16 10:15:48"", 1448, [[""page0"", ""2023..."
5281,user998,"[""2023-05-26 17:06:25"", 810, [[""page0"", ""2023-..."


### **Решения**

#### **Задача 1**

Ваша задача написать функцию `get_time_to_conversion`, которая принимает на вход сессии в формате, который вы определили для себя и подсчитывает среднее время между заходом на сайт и целевым действием.

Результат - число типа `float`, округленное до двух знаков после запятой, сохраните в переменную `res`

**Решение**

Напишите свое решение ниже

In [2]:
import json
import csv
from datetime import datetime
from collections import defaultdict

In [3]:
# функция чтения файлов\
def read_file(file):
  with open(file, newline='') as file_csv:
    reader = csv.reader(file_csv, delimiter=';')
    header = next(reader)
    users = [row for row in reader]
    return users

def parse_dates(obj: list) -> list:
  """
  Возвращает преобразованный список (строка заменена на формат datetime).

  Args:
      obj (list): список строк активности пользователя.

  Returns:
      list: Список сессий:
      первый элемент - время старта,
      второй элемент - общее количество времени, проведенное на сайте,
      третий элемент - список действий пользователя.
  """

  if isinstance(obj, list):
      return [parse_dates(item) for item in obj]
  elif isinstance(obj, str):
      try:
        return datetime.strptime(obj, '%Y-%m-%d %H:%M:%S')
      except ValueError:
        return obj
  return obj

def filter_data(sessions: list, conversion=True) -> list:
  """
  Возвращает список, разбитый по сессиям.

  Args:
      sessions (list): список строк активности пользователя.
  Kwargs:
      conversion (bool): определяет условие возврата массива -
      все активности или только результативные.

  Returns:
      list: Список сессий:
      первый элемент - пользователь,
      второй элемент - список сессий пользователя только с данными 'conversion'
      (результативное действие).
  """

  results = []
  for session in sessions:
      for user_data in session:
        try:
          user_id, json_part = user_data.split(',', 1)
          fixed_json = json_part.strip('"').replace('""', '"')
          fixed_json = fixed_json.replace('"[', '[').replace(']"', ']')

          # Сначала загружаем, потом обрабатываем
          user_info = json.loads(fixed_json)
          processed_info = parse_dates(user_info)  # Применяем кастомный парсер
          if conversion:
            # Фильтруем, оставляя только 'conversion'
            if isinstance(processed_info, list) and len(processed_info) > 2:
              conversion_data = [
                  item for item in processed_info[2]
                  if isinstance(item, list)
                  and item[0] == 'conversion'
                  ]
              processed_info[2] = conversion_data
            # В итоговом списке оставляем только данные с 'conversion'
            if len(processed_info[2]):
              results.append([user_id, processed_info])
          else:
            results.append([user_id, processed_info])

        except Exception as e:
          print(f"Ошибка обработки {user_id}: {e}")
  return results

def get_time_to_conversion(data: list) -> float:
  """
  Вычисляет среднее время, проведенное на сайте до результативных действий.

  Args:
      data (list): список действий пользователя:
      первый элемент - пользователь,
      второй элемент - список сессий пользователя только с данными 'conversion'
      (результативное действие).

  Returns:
      float: Среднее значение времени пользователя,
      проведенное от входа на сайт и до результативного действия.
  """
  result = []
  #преобразуем список списков в список значений
  for entry in data:
    user_id, details = entry
    # Первый элемент — дата, второй — число, третий — список событий
    first_date, number, events = details

    # Находим событие 'conversion' (если есть)
    conversion_time = None
    for event in events:
      if event[0] == 'conversion':
        conversion_time = event[1]
        break

    # Формируем упрощённую запись
    flattened_entry = [user_id, first_date, number, conversion_time]
    result.append(flattened_entry)

  #приозведем вычисления
  total_time = list(map(lambda x: (x[3] - x [1]).total_seconds(), result))
  return round(sum(total_time) / len(total_time), 2)

sessions = read_file('sessions.csv')
data_info = filter_data(sessions)
res = get_time_to_conversion(data_info)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res == 467.24
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `get_session_duration`, которая принимает на вход сессии в формате, который вы определили для себя и подсчитывает сколько вообще в среднем длится сессия.

Результат - число типа `float`, округленное до двух знаков после запятой, сохраните в переменную `res2`

**Решение**

Напишите свое решение ниже

In [ ]:

def get_session_duration(data_info: list) -> float:
  """
  Вычисляет среднее время, проведенное на сайте.

  Args:
      data_info (list): список действий пользователя:
      первый элемент - пользователь,
      второй элемент - список сессий пользователя.

  Returns:
      float: Среднее значение времени пользователя,
      проведенное на сайте.
  """
  # отбираем в список время, проведенное на сайте
  # (второй элемент вложенного списка)
  result = []
  for session in data_info:
    duration = session[1][1]
    result.append(duration)
  return round(sum(result) / len(result), 2)

data_info = filter_data(sessions, conversion=False)
res2 = get_session_duration(data_info)

901.35

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res2 == 901.35
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 3**

Напишите функцию `get_daily_session_count`, которая принимает на вход сессии в формате, который вы определили для себя и подсчитывает cколько разных заходов в течение дня делает пользователь.

Результат - число типа `float`, округленное до двух знаков после запятой, сохраните в переменную `res3`

**Решение**

Напишите свое решение ниже

In [9]:
def get_daily_session_count(data_info: list) -> float:
  """
  Вычисляет среднее количество заходов на сайт.

  Args:
      data_info (list): список действий пользователя:
      первый элемент - пользователь,
      второй элемент - список сессий пользователя (страница, дата).

  Returns:
      float: Среднее количество заходов на сайт в день.
  """

 # группируем сеансы по дням и количеством заходов каждого пользователя
  grouped_days = defaultdict(dict)

  for session in data_info:
    user = session[0]
    day = session[1][0].date()
    grouped_days[day][user] = grouped_days[day].get(user, 0) + 1

  # составляем словарь со средним значением заходов на сайт пользователей
  # за каждый день

  results = {}
  for day, session in grouped_days.items():
    results[day] = results.get(day, sum(session.values()) / len(session.values()))

  return round(sum(results.values()) / len(results), 2)

data_info = filter_data(sessions, conversion=False)

res3 = get_daily_session_count(data_info)

1.11

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [10]:
try:
    assert res3 == 1.11
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 4**

Напишите функцию `get_average_session_count`, которая принимает на вход сессии в формате, который вы определили для себя и подсчитывает сколько заходов вообще в среднем делает пользователь.

Результат - число типа `float`, округленное до двух знаков после запятой, сохраните в переменную `res4`

**Решение**

Напишите свое решение ниже

In [ ]:
def get_average_session_count(data_info):
  results = {}
  for session in data_info:
    user = session[0]
    results[user] = results.get(user, 0) + 1

  return round(sum(results.values()) / len(results), 2)



res4 = get_average_session_count(data_info)

5.28

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
try:
    assert res4 == 5.28
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
